<a href="https://colab.research.google.com/github/Bibhukalyan/langchain/blob/main/simpleLLM_langchain_expression_language(LCEL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.7/615.7 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
#Langsmith Tracing
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
groq_api_key = os.getenv('GROQ_API_KEY')

In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(groq_api_key=groq_api_key, model="Gemma2-9b-It")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x79fd7553a6b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x79fd755b45e0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage
messages=[
    SystemMessage(content="Translate the following text from English to French."),
    HumanMessage(content="Hello how are you?")
]

result=model.invoke(messages)

In [5]:
result

AIMessage(content="Bonjour, comment allez-vous ? \n\n\nThis is a more formal way of saying it. \n\nHere are some other options:\n\n* **Salut, ça va ?** (Informal)\n* **Coucou, comment vas-tu ?** (Very informal, used with friends) \n\n\n\nLet me know if you'd like to translate anything else!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 23, 'total_tokens': 99, 'completion_time': 0.138181818, 'prompt_time': 0.000140379, 'queue_time': 0.0154794, 'total_time': 0.138322197}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7bc82c99-6e5e-4d71-ab49-15ec45088665-0', usage_metadata={'input_tokens': 23, 'output_tokens': 76, 'total_tokens': 99})

In [6]:
from langchain_core.output_parsers import StrOutputParser
output_parser=StrOutputParser()
output_parser.invoke(result)

"Bonjour, comment allez-vous ? \n\n\nThis is a more formal way of saying it. \n\nHere are some other options:\n\n* **Salut, ça va ?** (Informal)\n* **Coucou, comment vas-tu ?** (Very informal, used with friends) \n\n\n\nLet me know if you'd like to translate anything else!\n"

In [7]:
## Using LCEL can chain the component

chain=model|output_parser
chain.invoke(messages)

'Bonjour, comment allez-vous ? \n\n\nYou can also say:\n\n* **Salut, ça va ?** (Informal)\n* **Comment vas-tu ?** (Informal, singular)\n'

In [9]:
##Prompt template
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"
prompt=ChatPromptTemplate.from_messages([
    ("system", generic_template),
    ("user", "{text}")
])

prompt.invoke({"language":"French", "text":"I love programming."})
chain=prompt|model|output_parser

In [11]:
result=chain.invoke({"language":"French", "text":"Hello how are you?"})
result

'Bonjour, comment allez-vous ? \n\n\nYou can also use:\n\n* **Salut, comment vas-tu ?** (More informal)\n\n'